<a href="https://colab.research.google.com/github/wtpgbaek/skinlesion/blob/main/labelme2voc_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function

import argparse
import glob
import os
import os.path as osp
import sys

!pip install matplotlib-venn
!apt-get -qq install -y libfluidsynth1
!pip install imgviz
import imgviz
import numpy as np
!pip install labelme
import labelme
from PIL import Image



E: Package 'libfluidsynth1' has no installation candidate


In [ ]:
def main():
    parser = argparse.ArgumentParser(
        formatter_class=argparse.ArgumentDefaultsHelpFormatter
    )
    parser.add_argument("input_dir", help="input annotated directory")
    parser.add_argument("output_dir", help="output dataset directory")
    parser.add_argument("--labels", help="labels file", required=True)
    parser.add_argument(
        "--noviz", help="no visualization", action="store_true"
    )
    args = parser.parse_args()

    if osp.exists(args.output_dir):
        print("Output directory already exists:", args.output_dir)
        sys.exit(1)
    os.makedirs(args.output_dir)
    os.makedirs(osp.join(args.output_dir, "JPEGImages"))
    os.makedirs(osp.join(args.output_dir, "SegmentationClass"))
    os.makedirs(osp.join(args.output_dir, "SegmentationClassPNG"))
    if not args.noviz:
        os.makedirs(
            osp.join(args.output_dir, "SegmentationClassVisualization")
        )
    print("Creating dataset:", args.output_dir)

    class_names = []
    class_name_to_id = {}
    for i, line in enumerate(open(args.labels).readlines()):
        class_id = i - 1  # starts with -1
        class_name = line.strip()
        class_name_to_id[class_name] = class_id
        if class_id == -1:
            assert class_name == "__ignore__"
            continue
        elif class_id == 0:
            assert class_name == "_background_"
        class_names.append(class_name)
    class_names = tuple(class_names)
    print("class_names:", class_names)
    out_class_names_file = osp.join(args.output_dir, "class_names.txt")
    with open(out_class_names_file, "w") as f:
        f.writelines("\n".join(class_names))
    print("Saved class_names:", out_class_names_file)

    for filename in glob.glob(osp.join(args.input_dir, "*.json")):
        print("Generating dataset from:", filename)

        label_file = labelme.LabelFile(filename=filename)

        base = osp.splitext(osp.basename(filename))[0]
        out_img_file = osp.join(args.output_dir, "JPEGImages", base + ".jpg")
        out_img_resized_file = osp.join(args.output_dir, "JPEGImages", base + "_resized.jpg")  # 수정된 부분
        out_lbl_file = osp.join(args.output_dir, "SegmentationClass", base + ".npy")
        out_png_file = osp.join(args.output_dir, "SegmentationClassPNG", base + ".png")
        if not args.noviz:
           out_viz_file = osp.join(
               args.output_dir,
               "SegmentationClassVisualization",
               base + ".jpg",
           )

    with open(out_img_file, "wb") as f:
        f.write(label_file.imageData)

    # Resize 이미지
    img = Image.open(out_img_file)
    img_resized = img.resize((128, 128), Image.ANTIALIAS)
    img_resized.save(out_img_resized_file)


    img = labelme.utils.img_data_to_arr(label_file.imageData)

    lbl, _ = labelme.utils.shapes_to_label(
            img_shape=img.shape,
            shapes=label_file.shapes,
            label_name_to_value=class_name_to_id,
        )
    labelme.utils.lblsave(out_png_file, lbl)
    np.save(out_lbl_file, lbl)

    if not args.noviz:
            viz = imgviz.label2rgb(
                lbl,
                imgviz.rgb2gray(img),
                font_size=15,
                label_names=class_names,
                loc="rb",
            )
            print(viz)
            print(viz.shape)
            imgviz.io.imsave(out_viz_file, viz)

In [ ]:
if __name__ == "__main__":
    # Colab에서 실행할 때, input_dir, output_dir, labels_file 수정
    input_dir = "/content/drive/MyDrive/cancer_data"
    output_dir = "/content/drive/MyDrive/lesion_output_dataset"
    labels_file = "/content/drive/MyDrive/lesion_labels.txt"

    # 스크립트 실행
    sys.argv = ["lebelme2voc.py", input_dir, output_dir, "--labels", labels_file]
    main()

Creating dataset: /content/drive/MyDrive/lesion_output_dataset
class_names: ('_background_', 'lesion')
Saved class_names: /content/drive/MyDrive/lesion_output_dataset/class_names.txt
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027305.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027306.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027307.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027308.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027309.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027311.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027312.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027313.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027317.json
Generating dataset from: /content/drive/MyDrive/cancer_data/ISIC_0027318.json
Generating dataset from: /content/dri

<ipython-input-10-ec3e830c5484>:67: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  img_resized = img.resize((128, 128), Image.ANTIALIAS)
